In [39]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
current_directory = os.getcwd()
print(current_directory) 



/Users/srite/repos/sri_avi_blog_posts/share_buybacks


In [159]:
### uncomment the following line if you dont have yfinance installed
#!pip install yfinance
#!pip install chart_studio
import yfinance as yf

import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go




In [5]:
pd.set_option('display.max_rows', 500)

# This Analysis will feature a 2 part analysis. 

## Part 1 -- do buyback announcements directly influence share prices?
#### Things to Look at
- 5, 30, 90 day share price avg pre and post annoucement
- 
- sdfkj

## Part 1

### Lets take a look at how 2023 and 2024 sharebuyback announcements immediately impacted share price by doing a pre vs post analysis

In [146]:
buybacks = pd.read_csv('buyback_announcements_2023_2024.csv')
buybacks.columns = buybacks.columns.str.replace(' ', '_')
buybacks.columns = buybacks.columns.str.replace('-', '_')

buybacks.head(5)

,Symbol,Company,Current_Price,Price_Change,Date,Percent_of_Shares,Buyback_Amount,Offer_Type,Buyback_Type,52_Week_Low,52_Week_High,Indicator(s)
0,ACN,Accenture,$371.27,2.90%,9/28/2023,2.00%,$4 billion,Open Market,Additional,$242.80,$375.73,Analyst Report - Analyst Revision - Gap Up -
1,ACM,AECOM,$89.23,1.30%,11/13/2023,8.30%,$1 billion,Open Market,NaN,$74.40,$94.01,Positive News -
2,A,Agilent Technologies,$132.01,-2.50%,1/9/2023,4.40%,$2 billion,Open Market,New,$96.80,$145.43,Upcoming Earnings - News Coverage - Gap Down -
3,ALKS,Alkermes,$28.91,-0.90%,2/15/2024,8.20%,$400 million,Open Market,New,$22.01,$33.71,Earnings Report - Analyst Downgrade - Analyst ...
4,MO,Altria Group,$40.90,1.10%,2/1/2024,1.40%,$1 billion,Open Market,New,$39.06,$48.04,Positive News -


In [147]:
#Get 5,30,90 day interval dates 
buybacks['Date'] = pd.to_datetime(buybacks['Date'])
buybacks['pre_date_5day'] = buybacks['Date']-timedelta(days=5)
buybacks['post_date_5day'] = buybacks['Date']+timedelta(days=5)
buybacks['pre_date_30day'] = buybacks['Date']-timedelta(days=30)
buybacks['post_date_30day'] = buybacks['Date']+timedelta(days=5)
buybacks['pre_date_90day'] = buybacks['Date']-timedelta(days=90)
buybacks['post_date_90day'] = buybacks['Date']+timedelta(days=90)


In [148]:
#Pull in share prices for dates in +- 90 days of Date using yfinance data


def fetch_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

stock_data = []

for index, row in buybacks.iterrows():
    ticker = row['Symbol']
    start_date = row['pre_date_90day']
    end_date = row['post_date_90day']
    announcement_date_price = row['Current_Price']
    buyback_date = row['Date']
    data_temp = fetch_stock_data(ticker, start_date, end_date)
    data_temp['Ticker'] = ticker
    data_temp['announcement_date'] = buyback_date;
    data_temp['announcement_date_price'] = announcement_date_price;
    
    stock_data.append(data_temp)

    
stock_data = pd.concat(stock_data)
stock_data = stock_data.reset_index()



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [149]:
stock_data['announcement_date_price'] = stock_data['announcement_date_price'].str.replace('$','')
stock_data['announcement_date_price'] = stock_data['announcement_date_price'].str.replace(',','')

stock_data['announcement_date_price'] = stock_data['announcement_date_price'].astype(float)

In [150]:
stock_data

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,announcement_date,announcement_date_price
0,2023-06-30,309.489990,312.690002,308.329987,308.579987,305.080139,2668500,ACN,2023-09-28,371.27
1,2023-07-03,308.000000,311.269989,307.410004,310.970001,307.443054,1172800,ACN,2023-09-28,371.27
2,2023-07-05,306.140015,311.980011,305.029999,309.899994,306.385193,1924800,ACN,2023-09-28,371.27
3,2023-07-06,306.920013,308.450012,304.649994,307.970001,304.477051,1375800,ACN,2023-09-28,371.27
4,2023-07-07,306.730011,311.260010,305.279999,305.760010,302.292145,1902800,ACN,2023-09-28,371.27
...,...,...,...,...,...,...,...,...,...,...
10726,2023-07-18,18.750000,19.700001,18.660000,19.510000,19.309624,3444600,VNO,2023-04-26,25.53
10727,2023-07-19,19.860001,20.549999,19.790001,20.530001,20.319147,3945700,VNO,2023-04-26,25.53
10728,2023-07-20,20.150000,20.379999,19.520000,20.049999,19.844076,5440600,VNO,2023-04-26,25.53
10729,2023-07-21,20.420000,20.600000,19.830000,20.309999,20.101406,2527600,VNO,2023-04-26,25.53


In [151]:
# Get announcement date share price
announcement_dates = stock_data[['Ticker', 'announcement_date']].drop_duplicates()
 
announcement_dates = pd.merge(announcement_dates, stock_data[['Date','Close','Ticker']], left_on=['Ticker','announcement_date'] , right_on=['Ticker','Date'])
announcement_dates

announcement_dates.rename(columns={'Close':'announcement_date_price_v2'}, inplace=True)



stock_data_merged = pd.merge(stock_data, announcement_dates[['Ticker','announcement_date_price_v2']], left_on =['Ticker'], right_on =['Ticker'])

stock_data_merged
#announcement_dates

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,announcement_date,announcement_date_price,announcement_date_price_v2
0,2023-06-30,309.489990,312.690002,308.329987,308.579987,305.080139,2668500,ACN,2023-09-28,371.27,300.769989
1,2023-07-03,308.000000,311.269989,307.410004,310.970001,307.443054,1172800,ACN,2023-09-28,371.27,300.769989
2,2023-07-05,306.140015,311.980011,305.029999,309.899994,306.385193,1924800,ACN,2023-09-28,371.27,300.769989
3,2023-07-06,306.920013,308.450012,304.649994,307.970001,304.477051,1375800,ACN,2023-09-28,371.27,300.769989
4,2023-07-07,306.730011,311.260010,305.279999,305.760010,302.292145,1902800,ACN,2023-09-28,371.27,300.769989
...,...,...,...,...,...,...,...,...,...,...,...
10603,2023-07-18,18.750000,19.700001,18.660000,19.510000,19.309624,3444600,VNO,2023-04-26,25.53,14.930000
10604,2023-07-19,19.860001,20.549999,19.790001,20.530001,20.319147,3945700,VNO,2023-04-26,25.53,14.930000
10605,2023-07-20,20.150000,20.379999,19.520000,20.049999,19.844076,5440600,VNO,2023-04-26,25.53,14.930000
10606,2023-07-21,20.420000,20.600000,19.830000,20.309999,20.101406,2527600,VNO,2023-04-26,25.53,14.930000


In [152]:
announcement_dates.head(5)

,Ticker,announcement_date,Date,announcement_date_price_v2
0,ACN,2023-09-28,2023-09-28,300.769989
1,ACM,2023-11-13,2023-11-13,80.470001
2,A,2023-01-09,2023-01-09,147.470001
3,ALKS,2024-02-15,2024-02-15,32.560001
4,MO,2024-02-01,2024-02-01,41.720001


In [153]:
stock_data_merged['deviation_from_announcement_date'] = (stock_data_merged['Close'] - stock_data_merged['announcement_date_price_v2'])
stock_data_merged['deviation_from_announcement_date_pct'] = round((stock_data_merged['Close'] - stock_data_merged['announcement_date_price_v2'])/stock_data_merged['announcement_date_price_v2'],3)



stock_data_merged['within_5_days'] = 0;
stock_data_merged['within_30_days'] = 0;
stock_data_merged['within_90_days'] = 0;


stock_data_merged.loc[abs( (stock_data_merged['Date'] - stock_data_merged['announcement_date']).dt.days) <= 5, 'within_5_days'] = 1
stock_data_merged.loc[abs( (stock_data_merged['Date'] - stock_data_merged['announcement_date']).dt.days) <= 30, 'within_30_days'] = 1
stock_data_merged.loc[abs( (stock_data_merged['Date'] - stock_data_merged['announcement_date']).dt.days) <= 90, 'within_90_days'] = 1
stock_data_merged['announcement_offset'] = (stock_data_merged['Date'] - stock_data_merged['announcement_date']).dt.days

In [154]:
stock_data_merged.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,announcement_date,announcement_date_price,announcement_date_price_v2,deviation_from_announcement_date,deviation_from_announcement_date_pct,within_5_days,within_30_days,within_90_days,announcement_offset
0,2023-06-30,309.489990,312.690002,308.329987,308.579987,305.080139,2668500,ACN,2023-09-28,371.27,300.769989,7.809998,0.026,0,0,1,-90
1,2023-07-03,308.000000,311.269989,307.410004,310.970001,307.443054,1172800,ACN,2023-09-28,371.27,300.769989,10.200012,0.034,0,0,1,-87
2,2023-07-05,306.140015,311.980011,305.029999,309.899994,306.385193,1924800,ACN,2023-09-28,371.27,300.769989,9.130005,0.030,0,0,1,-85
3,2023-07-06,306.920013,308.450012,304.649994,307.970001,304.477051,1375800,ACN,2023-09-28,371.27,300.769989,7.200012,0.024,0,0,1,-84
4,2023-07-07,306.730011,311.260010,305.279999,305.760010,302.292145,1902800,ACN,2023-09-28,371.27,300.769989,4.990021,0.017,0,0,1,-83


In [195]:
stock_data_merged.loc[stock_data_merged['Ticker'] == 'NVR',]

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,announcement_date,announcement_date_price,announcement_date_price_v2,deviation_from_announcement_date,deviation_from_announcement_date_pct,within_5_days,within_30_days,within_90_days,announcement_offset
6587,2023-11-16,6293.500000,6349.490234,6236.839844,6290.589844,6290.589844,25000,NVR,2024-02-14,7533.28,7501.72998,-1211.140137,-0.161,0,0,1,-90
6588,2023-11-17,6332.970215,6332.970215,6253.520020,6298.000000,6298.000000,23000,NVR,2024-02-14,7533.28,7501.72998,-1203.729980,-0.160,0,0,1,-89
6589,2023-11-20,6280.959961,6335.569824,6203.100098,6307.120117,6307.120117,22400,NVR,2024-02-14,7533.28,7501.72998,-1194.609863,-0.159,0,0,1,-86
6590,2023-11-21,6290.009766,6344.910156,6239.370117,6244.220215,6244.220215,20500,NVR,2024-02-14,7533.28,7501.72998,-1257.509766,-0.168,0,0,1,-85
6591,2023-11-22,6276.770020,6350.000000,6194.069824,6222.890137,6222.890137,12800,NVR,2024-02-14,7533.28,7501.72998,-1278.839844,-0.170,0,0,1,-84
6592,2023-11-24,6196.009766,6232.830078,6143.229980,6195.060059,6195.060059,6700,NVR,2024-02-14,7533.28,7501.72998,-1306.669922,-0.174,0,0,1,-82
6593,2023-11-27,6199.000000,6199.000000,6126.870117,6146.899902,6146.899902,14700,NVR,2024-02-14,7533.28,7501.72998,-1354.830078,-0.181,0,0,1,-79
6594,2023-11-28,6148.000000,6150.520020,6088.720215,6128.660156,6128.660156,24400,NVR,2024-02-14,7533.28,7501.72998,-1373.069824,-0.183,0,0,1,-78
6595,2023-11-29,6164.000000,6166.069824,6089.629883,6107.109863,6107.109863,22700,NVR,2024-02-14,7533.28,7501.72998,-1394.620117,-0.186,0,0,1,-77
6596,2023-11-30,6107.109863,6159.540039,6052.580078,6155.390137,6155.390137,29900,NVR,2024-02-14,7533.28,7501.72998,-1346.339844,-0.179,0,0,1,-76


In [156]:
# 30 day chart

stock_data_30day = stock_data_merged.loc[stock_data_merged['within_30_days'] == 1,]
stock_data_30day.head(50)
fig = px.line(stock_data_30day, x="announcement_offset", y="deviation_from_announcement_date_pct", color='Ticker')
fig.show()


In [166]:
## Add 30 day averages
#avg_30day = stock_data_30day
avg_30day_grouped = stock_data_30day[['Ticker', 'announcement_offset','deviation_from_announcement_date_pct']].groupby('announcement_offset')['deviation_from_announcement_date_pct'].mean()
avg_30day_grouped = avg_30day_grouped.to_frame()
avg_30day_grouped = avg_30day_grouped.reset_index()




,announcement_offset,deviation_from_announcement_date_pct
0,-30,-0.004574
1,-29,-0.005949
2,-28,-0.011978
3,-27,-0.009012
4,-26,-0.023211


In [192]:
# 30 day chart WITH average chart

stock_data_30day = stock_data_merged.loc[stock_data_merged['within_30_days'] == 1,]
stock_data_30day.head(50)
fig = px.line(stock_data_30day, x="announcement_offset", y="deviation_from_announcement_date_pct", color='Ticker')


## Add 30 day averages
#avg_30day = stock_data_30day
avg_30day_grouped = stock_data_30day[['Ticker', 'announcement_offset','deviation_from_announcement_date_pct']].groupby('announcement_offset')['deviation_from_announcement_date_pct'].mean()
avg_30day_grouped = avg_30day_grouped.to_frame()
avg_30day_grouped = avg_30day_grouped.reset_index()


## Add Chart element
trace1 = go.Scatter(
    x=avg_30day_grouped['announcement_offset'].tolist(),
    y=avg_30day_grouped['deviation_from_announcement_date_pct'].tolist(),
    line=dict(
        color='black',
        width = 5),
    name='Average'
    #xperiod="M1"
    #xperiodalignment="middle"
    #hovertemplate="%{y}%{_xother}"
)




fig.add_trace(trace1)


In [193]:
go.Figure(data=[trace1]).show()


In [157]:
# 5 day chart

stock_data_5day = stock_data_merged.loc[stock_data_merged['within_5_days'] == 1,]
fig = px.line(stock_data_5day, x="announcement_offset", y="deviation_from_announcement_date_pct", color='Ticker')
fig.show()


In [158]:
stock_data_merged.loc[stock_data_merged['Ticker'] == 'EXPE',]

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,announcement_date,announcement_date_price,announcement_date_price_v2,deviation_from_announcement_date,deviation_from_announcement_date_pct,within_5_days,within_30_days,within_90_days,announcement_offset
2736,2023-08-04,101.089996,104.230003,101.010002,103.190002,103.190002,5801200,EXPE,2023-11-02,139.1,94.839996,8.350006,0.088,0,0,1,-90
2737,2023-08-07,104.070000,108.489998,104.029999,107.489998,107.489998,5171500,EXPE,2023-11-02,139.1,94.839996,12.650002,0.133,0,0,1,-87
2738,2023-08-08,106.870003,110.339996,105.570000,109.839996,109.839996,3014100,EXPE,2023-11-02,139.1,94.839996,15.000000,0.158,0,0,1,-86
2739,2023-08-09,110.309998,111.970001,109.529999,109.809998,109.809998,3156300,EXPE,2023-11-02,139.1,94.839996,14.970001,0.158,0,0,1,-85
2740,2023-08-10,111.010002,112.570000,110.379997,111.669998,111.669998,2268600,EXPE,2023-11-02,139.1,94.839996,16.830002,0.177,0,0,1,-84
2741,2023-08-11,110.459999,112.470001,110.459999,111.540001,111.540001,2840500,EXPE,2023-11-02,139.1,94.839996,16.700005,0.176,0,0,1,-83
2742,2023-08-14,111.010002,112.059998,109.309998,111.930000,111.930000,1795700,EXPE,2023-11-02,139.1,94.839996,17.090004,0.180,0,0,1,-80
2743,2023-08-15,111.980003,112.330002,106.660004,107.430000,107.430000,3225700,EXPE,2023-11-02,139.1,94.839996,12.590004,0.133,0,0,1,-79
2744,2023-08-16,107.050003,108.360001,106.790001,107.139999,107.139999,1989200,EXPE,2023-11-02,139.1,94.839996,12.300003,0.130,0,0,1,-78
2745,2023-08-17,108.000000,108.000000,105.580002,105.629997,105.629997,3045800,EXPE,2023-11-02,139.1,94.839996,10.790001,0.114,0,0,1,-77


In [30]:
## read in yfinance data for stocks

aapl = yf.Ticker('AAPL')
aapl_df = aapl.history(period='1y')
#aapl_df['Close'].plot(title="APPLE's stock")
aapl_df = aapl_df.reset_index()
aapl_df



,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-02-23 00:00:00-05:00,149.293135,149.541808,146.458276,148.606796,48394200,0.00,0.0
1,2023-02-24 00:00:00-05:00,146.328931,146.408508,144.946312,145.931061,55469600,0.00,0.0
2,2023-02-27 00:00:00-05:00,146.925767,148.378007,146.667138,147.134644,44998500,0.00,0.0
3,2023-02-28 00:00:00-05:00,146.269261,148.288481,146.050427,146.627350,50547000,0.00,0.0
4,2023-03-01 00:00:00-05:00,146.050432,146.448302,144.240088,144.538498,55479000,0.00,0.0
5,2023-03-02 00:00:00-05:00,143.613439,145.931070,143.135977,145.135315,52238100,0.00,0.0
6,2023-03-03 00:00:00-05:00,147.253989,150.307696,146.547767,150.228119,70732300,0.00,0.0
7,2023-03-06 00:00:00-05:00,152.973464,155.470147,152.645229,153.013260,87558000,0.00,0.0
8,2023-03-07 00:00:00-05:00,152.883962,153.212211,150.327614,150.795120,56182000,0.00,0.0
9,2023-03-08 00:00:00-05:00,151.998686,152.655185,151.023893,152.058365,47204800,0.00,0.0
